In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

# Now import tensorflow/keras modules
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import os 
import re

image_dir = 'BBBC005_v1_images'
files = []
labels = []

for file in os.listdir(image_dir):
    if file.endswith('.TIF'):
        files.append(os.path.join(image_dir, file))
        labels.append(int(re.findall('_C(.*?)_', file)[0]))
dataset = dc.data.ImageDataset(files, np.array(labels))


In [4]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [5]:
features = tf.keras.Input(shape = (520,696,1))
prev_layer = features
for num_outputs in [16,32,64,128,256]:
    prev_layer = layers.Conv2D(num_outputs, kernel_size=5, strides=2,
                               activation=tf.nn.relu)(prev_layer)
output = layers.Dense(1)(layers.Flatten()(prev_layer))

keras_model = tf.keras.Model(inputs=features, outputs=output)
learning_rate = dc.models.optimizers.ExponentialDecay(0.001, 0.9, 250)
model = dc.models.KerasModel(
    keras_model,
    loss = dc.models.losses.L2Loss(),
    learning_rate = learning_rate,
    model_dir='model'
)



In [6]:
model.restore()

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


In [7]:
y_pred = model.predict(test_dataset).flatten()
print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))


56.404032559677376


In [8]:
#model.fit(train_dataset, nb_epoch=50)